## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import hvplot.pandas
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [3]:
# Check for outliers
boxplot = application_df.hvplot.box(y='ASK_AMT', by='IS_SUCCESSFUL', height=400, width=400, legend=True, hover=True)
boxplot

:BoxWhisker   [IS_SUCCESSFUL]   (ASK_AMT)

In [4]:
application_df.sort_values(by="ASK_AMT", ascending=False).head(10)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
33175,942854057,IHC HEALTH SERVICES INC,T3,Independent,C1000,Heathcare,Trust,1,50M+,N,8597806340,1
34222,990073480,TRUSTEES OF THE ESTATE OF BERNICE PAUAHI BISHOP,T3,Independent,C2000,Preservation,Co-operative,1,50M+,N,8556638692,0
33678,946174066,STANFORD HEALTH CARE,T3,Independent,C1200,Heathcare,Trust,1,50M+,N,5591584994,0
24795,741109620,WILLIAM MARSH RICE UNIVERSITY,T3,Independent,C2000,Preservation,Trust,1,50M+,N,4653011914,0
31337,900779996,CHILDRENS HEALTHCARE OF ATLANTA INC,T3,CompanySponsored,C1000,Preservation,Trust,1,50M+,N,3391919220,1
24947,742568941,HEALTHONE,T3,Independent,C1000,Heathcare,Trust,1,50M+,N,2310256039,1
21633,590747311,SOUTHERN BAPTIST HOSPITAL OF FLORI DA INC,T3,Independent,C1000,Heathcare,Trust,1,50M+,N,2264109450,0
33027,941156617,PRESIDENT-BOARD OF TRUSTEES SANTA CLARA COLLEGE,T3,Independent,C2000,Preservation,Trust,1,50M+,N,1893400128,0
773,131788491,AMERICAN CANCER SOCIETY INC,T3,Family/Parent,C2800,Preservation,Trust,1,50M+,N,1736232349,1
33256,943120807,EDUCATIONAL CORPORATION MUKOGAWA GAKUIN,T3,Independent,C2000,Preservation,Trust,1,50M+,N,1665460552,0


In [5]:
# Delete outliers
application_df = application_df.drop(axis=0, index=[33175,34222,33678,24795])

In [6]:
# Look at STATUS value counts for binning
application_df["STATUS"].value_counts()

STATUS
1    34290
0        5
Name: count, dtype: int64

In [7]:
# Look at SPECIAL_CONSIDERATIONS value counts for binning
application_df["SPECIAL_CONSIDERATIONS"].value_counts()

SPECIAL_CONSIDERATIONS
N    34268
Y       27
Name: count, dtype: int64

In [8]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME","STATUS","SPECIAL_CONSIDERATIONS"], axis=1)

In [9]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8743
IS_SUCCESSFUL          2
dtype: int64

In [10]:
# Look at AFFILIATION value counts for binning
application_df["AFFILIATION"].value_counts()

AFFILIATION
Independent         18476
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64

In [11]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `AFFILIATION_to_replace`
AFFILIATION_to_replace = ["Family/Parent", "National", "Regional"]

# Replace in dataframe
for app in AFFILIATION_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(app,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

AFFILIATION
Independent         18476
CompanySponsored    15705
Other                 114
Name: count, dtype: int64

In [12]:
# Look at USE_CASE value counts for binning
application_df["USE_CASE"].value_counts()

USE_CASE
Preservation     28093
ProductDev        5671
CommunityServ      384
Heathcare          144
Other                3
Name: count, dtype: int64

In [13]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `USE_CASE_to_replace`
USE_CASE_to_replace = ["CommunityServ", "Heathcare"]

# Replace in dataframe
for app in USE_CASE_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

USE_CASE
Preservation    28093
ProductDev       5671
Other             531
Name: count, dtype: int64

In [14]:
# Look at ORGANIZATION value counts for binning
application_df["ORGANIZATION"].value_counts()

ORGANIZATION
Trust           23512
Association     10255
Co-operative      485
Corporation        43
Name: count, dtype: int64

In [15]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `ORGANIZATION_to_replace`
ORGANIZATION_to_replace = ["Co-operative", "Corporation"]

# Replace in dataframe
for app in ORGANIZATION_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(app,"Other")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

ORGANIZATION
Trust          23512
Association    10255
Other            528
Name: count, dtype: int64

In [16]:
# Look at INCOME_AMT value counts for binning
application_df["INCOME_AMT"].value_counts(sort=False)

INCOME_AMT
0                24388
1-9999             728
10000-24999        543
100000-499999     3374
10M-50M            240
25000-99999       3747
50M+               135
1M-5M              955
5M-10M             185
Name: count, dtype: int64

In [17]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27033
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [18]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "66", "T12", "T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27033
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [19]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17325
C2000     6072
C1200     4836
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [20]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts().loc[lambda x: x>1]

CLASSIFICATION
C1000    17325
C2000     6072
C1200     4836
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [21]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# since there are more to replace than to keep, we create a list of values to keep
classifications_to_keep = ["C1000", "C2000", "C1200", "C3000", "C2100"]
classifications_to_replace = application_df[~application_df['CLASSIFICATION'].isin(classifications_to_keep)]
classifications_to_replace = classifications_to_replace['CLASSIFICATION'].value_counts().index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17325
C2000     6072
C1200     4836
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [22]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34295 entries, 0 to 34298
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   APPLICATION_TYPE  34295 non-null  object
 1   AFFILIATION       34295 non-null  object
 2   CLASSIFICATION    34295 non-null  object
 3   USE_CASE          34295 non-null  object
 4   ORGANIZATION      34295 non-null  object
 5   INCOME_AMT        34295 non-null  object
 6   ASK_AMT           34295 non-null  int64 
 7   IS_SUCCESSFUL     34295 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 2.4+ MB


In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
APPLICATION_TYPE_dummies = pd.get_dummies(application_df['APPLICATION_TYPE'], dtype=int)
AFFILIATION_dummies = pd.get_dummies(application_df['AFFILIATION'], dtype=int)
CLASSIFICATION_dummies = pd.get_dummies(application_df['CLASSIFICATION'], dtype=int)
USE_CASE_dummies = pd.get_dummies(application_df['USE_CASE'], dtype=int)
ORGANIZATION_dummies = pd.get_dummies(application_df['ORGANIZATION'], dtype=int)
INCOME_AMT_dummies = pd.get_dummies(application_df['INCOME_AMT'], dtype=int)
# SPECIAL_CONSIDERATIONS_dummies = pd.get_dummies(application_df['SPECIAL_CONSIDERATIONS'], dtype=int)

In [24]:
# create 'numeric_application_df' and drop all categorical columns
numeric_application_df = application_df.drop(columns=['APPLICATION_TYPE', 'AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT'], axis=1)

In [25]:
# concate categorical dummies with application_df
processed_application_df = pd.concat([numeric_application_df,APPLICATION_TYPE_dummies,AFFILIATION_dummies,CLASSIFICATION_dummies,USE_CASE_dummies,ORGANIZATION_dummies,INCOME_AMT_dummies], axis=1)

In [26]:
# Split our preprocessed data into our features and target arrays
 # features
X = processed_application_df.drop('IS_SUCCESSFUL', axis=1)
 # target
y = processed_application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [27]:
X.shape

(34295, 34)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
X_train_scaled.shape, y_train.shape, X_test_scaled.shape, y_test.shape

((25721, 34), (25721,), (8574, 34), (8574,))

## Compile, Train and Evaluate the Model

In [30]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=1, max_value=80)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    activation = hp.Choice('activation', values=['relu','tanh'])
    optimizer = hp.Choice("optimizer", values=["sgd","adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
        
    model = tf.keras.Sequential()
    
    # Allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=80,
        step=1), activation=activation, input_dim=34))
    
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation=activation))
        
    # Output layer
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [31]:
import keras_tuner as kt
random_search_tuner = kt.RandomSearch(
    create_model, objective="val_accuracy", max_trials=5, overwrite=True, directory="ASoup_funding_choice_1", project_name="funding_choice_1", seed=78)

random_search_tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test))

Trial 5 Complete [00h 00m 40s]
val_accuracy: 0.7281315326690674

Best val_accuracy So Far: 0.7281315326690674
Total elapsed time: 00h 02m 56s


In [32]:
top3_hp = random_search_tuner.get_best_hyperparameters(3)
best_hp = top3_hp[0]
best_hp.values

{'n_hidden': 1,
 'n_neurons': 79,
 'learning_rate': 0.008316141310476274,
 'activation': 'tanh',
 'optimizer': 'adam',
 'first_units': 20}

In [33]:
# Evaluate the top model
best_trial = random_search_tuner.oracle.get_best_trials(1)[0]
best_trial.summary()


Trial 4 summary
Hyperparameters:
n_hidden: 1
n_neurons: 79
learning_rate: 0.008316141310476274
activation: tanh
optimizer: adam
first_units: 20
Score: 0.7281315326690674


In [34]:
best_trial.metrics.get_last_value("val_accuracy")

0.7281315326690674

In [35]:
best_model = random_search_tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
# model_loss, model_accuracy = top_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5729 - accuracy: 0.7281 - 110ms/epoch - 410us/step
Loss: 0.5728791356086731, Accuracy: 0.7281315326690674


In [36]:
# Export our model to HDF5 file
best_model.save('ASoup_Optimize_ReduceData.h5')

/Users/jiajialiu/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
